In [158]:
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='ionab10', api_key='')

from collections import Counter
import statsmodels.api as sm
import statsmodels.genmod.families.links as links

from scipy.stats import poisson

from math import log,exp,factorial
import pandas as pd
from numpy import mean

In [365]:
freq=pd.DataFrame()

works=pd.DataFrame([('SHAKESPEARE_Complete_Works.txt','Shakespeare'),
                  ('CERVANTES_Don_Quixote.txt','DonQuixote'),
                  ('TOLSTOY_War_and_Peace.txt','War&Peace')
                 ],
                  columns=['filename','tag'])
works.set_index('tag', inplace=True)

for tag,row in works.iterrows():
    
    with open('./{}'.format(row['filename']),'r') as f:
        text = f.read()
        
    works.set_value(tag,'wordcount',len(text.split()))
    freq_counter=Counter([len(word) for word in text.split()])

    for item in freq_counter.items():
        freq.set_value(item[0],tag,item[1])
    freq.fillna(0, inplace=True)
        
    mu=sum([i*item for i,item in freq[tag].iteritems()])/sum(freq[tag])
    works.set_value(tag,'mu',mu)
    
freq.sort_index(inplace=True)
freq['ALL']=[sum(row) for i,row in freq.iterrows()]

works.set_value('ALL','wordcount',sum(freq['ALL']))
works.set_value('ALL','mu',sum([i*item for i,item in freq['ALL'].iteritems()])/sum(freq['ALL']))

works

,filename,wordcount,mu
tag,,,
Shakespeare,SHAKESPEARE_Complete_Works.txt,902892.0,4.483688
DonQuixote,CERVANTES_Don_Quixote.txt,428827.0,4.365017
War&Peace,TOLSTOY_War_and_Peace.txt,566206.0,4.669663
ALL,NaN,1897925.0,4.512356


In [366]:
freq.head()

,Shakespeare,DonQuixote,War&Peace,ALL
1,35512.0,12977.0,14043.0,62532.0
2,138781.0,79975.0,87778.0,306534.0
3,173521.0,98428.0,133919.0,405868.0
4,188973.0,80207.0,92899.0,362079.0
5,121331.0,45612.0,61405.0,228348.0


In [368]:
data=[]
annotations={}

for tag in works.index:

    actual=go.Scatter(
        x = list(freq.index),
        y = list(freq[tag]),
        mode = 'markers',
        name ='actual'
    )

    predicted=go.Scatter(
        x = list(freq.index),
        y = [sum(freq[tag])*poisson.pmf(k,mu = works.loc[tag]['mu']) for k in freq.index],
        name='predicted'
    )

    data = data+[actual,predicted]
    
    annotations[tag]=[
        go.Annotation(x=0.5,
            y=1,
            showarrow=False,
            text='$\lambda = {}$'.format(works.loc[tag]['mu']),
            xref='paper',
            yref='paper')
    ]

updatemenus = list([
    dict(buttons=[dict(label = tag,
                       method = 'update',
                       args = [{'visible': [True if j==2*i or j==2*i+1 else False for j in range(len(data))]},
                               {'title':'Word-length Frequency: {}'.format(tag),
                                'annotations':annotations[tag]
                               }]) 
                  for i,tag in enumerate(works.index)]
        )
])

layout=go.Layout(
    title='Word-length Frequency: {}'.format(tag),
    annotations=go.Annotations(annotations[tag]),
    xaxis=dict(title='Word-length'),
    yaxis=dict(title='Frequency'),
    updatemenus=updatemenus
)

fig=go.Figure(data=data,layout=layout)

# Plot and embed in ipython notebook!
py.iplot(fig, filename='WLF')

In [269]:
model = sm.GLM(freq[name],list(freq.index), family=sm.families.Poisson())
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            Shakespeare   No. Observations:                   36
Model:                            GLM   Df Residuals:                       35
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -8.5967e+06
Date:                Wed, 27 Dec 2017   Deviance:                   1.7193e+07
Time:                        18:04:06   Pearson chi2:                 4.49e+10
No. Iterations:                    12                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2617   8.18e-05   3200.720      0.000       0.262       0.262
==============================================================================
"""